# Overview

This notebook is the development notes for using `bluesky` and `apstools` to conduct Tomography experiment at 6BM-A at the advanced photon source.

The notebook contains the following sections:
* preparation
    * installation and configuration of `bluesky`+`apstools` for 6BM-A
        * package installation
        * meta-data broker configuration
    * user-define functions that is useful for the tomography experiment
* device declaration: declare `ophyd` interface for `Epics` devices
    * device initilization
* tomography experiment
    * plan generation
    * dry-run
    * connect to devices for experiment

# Preparation

Install with `pip`
------------------

__WARNING__: it is recommended to use conda to install bluesky related pacakge from now.

Install `bluesky` and `apstools` with `pip` (not recommended due to dependency issues):

```bash
pip install -U pip
pip install boltons mongoquery pims pyepics pyRestTable tzlocal jupyter suitcase matplotlib
pip install git+https://github.com/Nikea/historydict#egg=historydict \
            git+https://github.com/NSLS-II/amostra#egg=amostra \
            git+https://github.com/NSLS-II/bluesky#egg=bluesky \
            git+https://github.com/NSLS-II/databroker#egg=databroker \
            git+https://github.com/NSLS-II/doct#egg=doct \
            git+https://github.com/NSLS-II/event-model#egg=event_model \
            git+https://github.com/NSLS-II/ophyd#egg=ophyd \
            git+https://github.com/NSLS-II/hklpy#egg=hklpy
pip install apstools
```

Install with `conda`
--------------------

Install bluesky core packages first
```bash
conda install bluesky -c lightsource2-tag
```

then the apstools dependencies
```bash
conda install pyresttable -c prjemian
```

followed by installing apstools
```bash
conda install apstools -c aps-anl-dev
```

To avoid the warning/runtime-error when running RE, it is recommended to add a file named __pinned__ to the _conda-meta/_ dierectory for given environment with the following content
```shell
tornado<5
```
followed by update the tornado package through conda using 
```bash
conda update tornado
```

> If the command above does not downgrade tornado properly, one can use `conda install tornado==4.5.3` to force reinstall a lower version.

Finally, install jupyter and matplotlib
```bash
conda install jupyter matplotlib
```

> alternatively, one can setup a `.conda.rc` file in the home directory to specify the channels
```
channels:
    - defaults
    - conda-forge
    - lightsource2-tag
    - lightsource2-dev
    - aps-anl-tag
    - aps-anl-dev
    - prjemian
```
> It is important to notice that this configuration file is for the current user, shared by all related environments.

Then configure MongoDB for meta-data handling

The following YAML config file should be place under `~/.config/` to use MongoDB as meta-data handler.

```yml
# ~/.config/databroker/mongodb_config.yml

description: 'heavyweight shared database'
metadatastore:
   module: 'databroker.headersource.mongo'
   class: 'MDS'
   config:
       host: 'otz.aps.anl.gov'
       port: 27017
       database: 'metadatastore-production-v1'
       timezone: 'US/Central'
assets:
   module: 'databroker.assets.mongo'
   class: 'Registry'
   config:
       host: 'otz.aps.anl.gov'
       port: 27017
       database: 'filestore-production-v1'
```

In [1]:
# initialize the data base
from databroker import Broker
db = Broker.named("mongodb_config")

# subscribe both mongodb and callback to runtime engine
import bluesky
from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback

RE = RunEngine({})
RE.subscribe(db.insert)
RE.subscribe(BestEffortCallback())

/home/beams/S6BM/opt/miniconda3/envs/bluesky_py36/lib/python3.6/site-packages/databroker/_core.py:962: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(f)


1

In [2]:
import socket
import getpass

socket.gethostname()
getpass.getuser()

's6bm'

In [3]:
# metadata for diagnostics
import os
from datetime import datetime
import apstools
import ophyd
import socket
import getpass

HOSTNAME = socket.gethostname() or 'localhost'
USERNAME = getpass.getuser() or '6-BM-A user'
RE.md['beamline_id'] = 'APS 6-BM-B'
RE.md['proposal_id'] = 'internal test'
RE.md['pid'] = os.getpid()
RE.md['login_id'] = USERNAME + '@' + HOSTNAME
RE.md['BLUESKY_VERSION'] = bluesky.__version__
RE.md['OPHYD_VERSION'] = ophyd.__version__
RE.md['apstools_VERSION'] = apstools.__version__
RE.md['SESSION_STARTED'] = datetime.isoformat(datetime.now(), " ")

In [4]:
# user defined function goes here
from pprint import pprint


# 1. checking the APS beam status
import apstools.devices as APS_devices

aps = APS_devices.ApsMachineParametersDevice(name="APS")

pprint(aps)

ApsMachineParametersDevice(prefix='', name='APS', read_attrs=['current', 'lifetime', 'machine_status', 'operating_mode', 'shutter_permit', 'fill_number', 'orbit_correction', 'global_feedback', 'global_feedback_h', 'global_feedback_v', 'operator_messages', 'operator_messages.operators', 'operator_messages.floor_coordinator', 'operator_messages.fill_pattern', 'operator_messages.last_problem_message', 'operator_messages.last_trip_message', 'operator_messages.message6', 'operator_messages.message7', 'operator_messages.message8'], configuration_attrs=['operator_messages'])


Now from the hardware end, we need additional setup IOC (soft) for this status, recommended method `VME`

Here is a quick example from 2BM
```python
instrument_in_use = EpicsSignalRO(
    "2bm:instrument_in_use", 
    name="instrument_in_use")

def operations_in_2bmb():
    """returns True if allowed to use X-ray beam in 2-BM-B station"""
    v = instrument_in_use.value
    enums = instrument_in_use.enum_strs
    return enums[v] == "2-BM-B"
```

In [5]:
# 2. assume we have the same setup describe above

from ophyd import EpicsSignalRO

instrument_in_use = EpicsSignalRO("6bm:instrument_in_use", name="instrument_in_use")

RE.md['INSTRUMENT_IN_USE'] = instrument_in_use.get()

In [6]:
print(instrument_in_use)

EpicsSignalRO(read_pv='6bm:instrument_in_use', name='instrument_in_use', value=1, timestamp=1554729382.179483, pv_kw={}, auto_monitor=False, string=False)


In [7]:
# 3. check ambien light in hutch
import apstools.synApps_ophyd

# grab all calcs
calcs = apstools.synApps_ophyd.userCalcsDevice("6bma1:", name="calcs", )

# calc1.ch8 is ambient light checker
#calc1 = calcs.calc1
hutch_light_on = bool(calcs.calc1.val.get())

print(hutch_light_on)

True


In [8]:
print(calcs.calc1.val.get() ,hutch_light_on)

1.0 True


In [9]:
# 4. define the ENV VAR denote the experiment nature
# NOTE:
#    the return value of instrument_in_use is not yet decided

# conducting experiment mode
in_production = aps.inUserOperations \
            and (instrument_in_use.get() in (1, "6-BM-A")) \
            and (not hutch_light_on)

# testing mode, supercede in_production
in_dryrun = True

In [10]:
print(in_production, aps.inUserOperations)

False False


# Device declaration

In [11]:
# necessary import
import apstools.devices as APS_devices

## shutter

In [12]:
from bluesky.suspenders import SuspendFloor

if in_production:
    # define the real shutter used at 6BMA@APS
    # NOTE: 
    #   this requires connection to the hardware, otherwise a connection error will be raised

    A_shutter = APS_devices.ApsPssShutterWithStatus(
            "6bmb1:rShtrA:",
            "PA:06BM:STA_A_FES_OPEN_PL",
            name="A_shutter",
        )
    A_shutter.pss_state
    # no scans until A_shutter is open
    suspend_A_shutter = SuspendFloor(A_shutter.pss_state, 1)
    #suspend_A_shutter.install(RE)
#     RE.install_suspender(suspend_A_shutter)
    
    # no scans if aps.current is too low
    suspend_APS_current = SuspendFloor(aps.current, 2, resume_thresh=10)
    RE.install_suspender(suspend_APS_current)

else:
    # first, a simulated shutter to demonstrate the design of epics controled shutter

    A_shutter = APS_devices.SimulatedApsPssShutterWithStatus(name="A_shutter")
    suspend_A_shutter = SuspendFloor(A_shutter.pss_state, 1)
    

In [13]:
RE.suspenders

()

To open a shutter, use `A_shutter.open`. 
Similarly, the shutter can be closed with `A_shutter.close`.
One can also check the current status of the shutter using `A_shutter.isOpen`.

## area detector

In [14]:
import os
import datetime
from pathlib import Path, PureWindowsPath

# production control ENV vars
ADPV_prefix = "1idPG2"   # AreaDetector prefix
# OUTPUT_ROOT = "/home/beams/S6BM/user_data"
OUTPUT_ROOT = "Y:\\"

# -----------------------
# -- user config block --
# -----------------------
CYCLE = "2019-1"
EXPID = "internal_apr19"
USER  = EXPID
SAMPLE = "test"

FILE_PATH = str(PureWindowsPath(Path("/".join([OUTPUT_ROOT, CYCLE, EXPID, "tomo", SAMPLE])+"/")))+'\\'
# FILE_PATH =  os.path.join(OUTPUT_ROOT, CYCLE, EXPID, "tomo", SAMPLE) + os
FILE_PREFIX = SAMPLE

# show where files will be stored
print(FILE_PATH)
print(FILE_PREFIX)

Y:\2019-1\internal_apr19\tomo\test\
test


In [15]:
# As of today, there is still no support for Python class enclosure, so we have to make a class for the detector
from ophyd import AreaDetector
from ophyd import SingleTrigger
from ophyd import ADComponent
from ophyd import PointGreyDetectorCam
from ophyd import ProcessPlugin
from ophyd import TIFFPlugin
from ophyd import sim

# class PointGreyDetectorCam6BM(PointGreyDetectorCam):
#     """Extend the default Point Grey detector cam"""
#     frame_rate_on_off = ADComponent(EpicsSignalWithRBV, "FrameRateOnOff")


class PointGreyDetector6BM(SingleTrigger, AreaDetector):
    """Point Gray area detector used at 6BM"""
    # cam component
    cam = ADComponent(PointGreyDetectorCam, "cam1:")
    
    # proc plugin
    proc1 = ADComponent(ProcessPlugin, suffix="Proc1:")
    
    # tiff plugin
    tiff1 = ADComponent(
        TIFFPlugin,
        suffix="TIFF1:",
#         root=OUTPUT_ROOT,                       # for databroker
#         write_path_template=FILE_PATH,          # for EPICS AD
    )
    

# ------------------------------------
# ----- Instantiate the detector ----- 
# ------------------------------------

# Area Detector (AD) config block
config_cam = {
    "num_images":     1,           # number of images (nFrame)
    "image_mode":     "Multiple",  #
    "trigger_mode":   "Internal",  #
    "acquire_time":   0.05,           # exposure time (fExposureTime)
    "acquire_period": 0.05+0.01,      #
    "gain":           5,           # detector gain [0~30]
}

config_proc1 = {
    "enable":           1,  # toggle on proc1
    "enable_filter":    1,  # enable filter
    "num_filter":       5,  # change number_filtered in proc1 (same as nFrame)
    "reset_filter":     1,  # reset number_filtered
}

config_tiff1 = {
    "nd_array_port":    "PROC1",      # switch port for TIFF plugin
    "file_write_mode":  "Stream",     # change write mode
    "auto_save":        "Yes",        # turn on file save
    "file_path":        FILE_PATH,    # set file path
    "file_name":        FILE_PREFIX,  # img name prefix
}


if in_production or in_dryrun:
    pg2_det = PointGreyDetector6BM("1idPG2:", name='pg2_det')
    pg2_det.read_attrs.append('tiff1')  # this is very important
    
    # catch timeout error in case detector not responding
    try:
        for k, v in config_cam.items():     pg2_det.cam.stage_sigs[k]   = v
        for k, v in config_proc1.items():   pg2_det.proc1.stage_sigs[k] = v
        for k, v in config_tiff1.items():   pg2_det.tiff1.stage_sigs[k] = v
    except TimeoutError as _exc:
        print(f"{_exc}\n !! Could not connect with area detector {pg2_det}")
    
else:
    pg2_det = sim.noisy_det  # use ophyd simulated detector


## motor

In [16]:
from ophyd import MotorBundle
from ophyd import Component
from ophyd import EpicsMotor

# NOTE: 
#    the PV for actual motors is still unknown
class TomoStage(MotorBundle):
    #rotation
    preci = Component(EpicsMotor, "6bmpreci:m1",
                      name='preci',
                     )
    
    samX = Component(EpicsMotor, "6bma1:m19",
                     name='samX',
                    )
    
    samY = Component(EpicsMotor, "6bma1:m18",
                     name="samY",
                    )

if in_production or in_dryrun:
    tomostage = TomoStage(name='tomostage')

    samx  = tomostage.samX
    samy  = tomostage.samY
    preci = tomostage.preci
    
    dummy = sim.motor

else:
    tomostage = MotorBundle()
    tomostage.preci = sim.motor
    tomostage.samX = sim.motor
    tomostage.samY = sim.motor


# Tomography experiment

## Scan plans

Define the scan parameters within plan functions

In [17]:
# ---------------------------
# ----- Scan parameters -----
# ---------------------------
n_white        =  10
n_dark         =  10
samOutDist     = -5.00              # mm
omega_step     =  0.25              # degrees
acquire_time   =  0.05              # sec
acquire_period = acquire_time+0.01  # sec
time_wait      = acquire_period*2   # sec
omega_start    =  -180
omega_end      =  180
n_frames       =  5                 # proc.n_filters, cam.n_images

number_of_projections = int(abs(omega_end-omega_start)/omega_step)+1  # 0 ~ 180

test_mode = True  # skip wait time during plan summarize 

In [18]:
# ---------------------------------------
# ----- base detector configuration ----- 
# ---------------------------------------

# NOTE:
# -- gradually building a comprehensive list of options for the configuration
# -- use 

# Area Detector (AD) config block
config_cam = {
    "num_images":     n_frames,           # number of images (nFrame)
    "image_mode":     "Multiple",  #
    "trigger_mode":   "Internal",  #
    "acquire_time":   0.05,        # exposure time (fExposureTime)
    "acquire_period": 0.05+0.01,   #
    "gain":           5,           # detector gain [0~30]
}

config_proc1 = {
    "enable":           1,  # toggle on proc1
    "enable_filter":    1,  # enable filter
    "num_filter":       n_frames,  # change number_filtered in proc1 (same as nFrame)
    "reset_filter":     1,  # reset number_filtered
}

config_tiff1 = {
    "nd_array_port":    "PROC1",      # switch port for TIFF plugin
    "file_write_mode":  "Stream",     # change write mode
    "auto_save":        "Yes",        # turn on file save
    "file_path":        FILE_PATH,    # set file path
    "file_name":        FILE_PREFIX,  # img name prefix
}

In [19]:
from ophyd.status import DeviceStatus

import numpy as np

import bluesky.plans         as bp
import bluesky.preprocessors as bpp
import bluesky.plan_stubs    as bps

def cleanup():
    pass


def tiff_io_wait():
    """
    mimic io wait for the tiff plugin
    
    while (epics_get("1idPG2:TIFF1:Capture_RBV") != "Done") {
        sleep(0.05)
        }
    """
    while (pg2_det.tiff1.capture.get() != "Done"):
        yield from bps.sleep(time_wait)


def cam_io_wait():
    """
    mimicing the waiting scheme in original epics script
    
    while ( (epics_get("1idPG2:cam1:NumImages_RBV")-epics_get("1idPG2:cam1:NumImagesCounter_RBV")) != 0 ) {
            sleep(fExposureTime + trigger_wait)
            trigger_count++
            if ( (time()-tic) > 30 ) {
                sendemailalert "Pointgray Acquire timeout"
                sleep(600)
            }
        }

    """
    _st = DeviceStatus(pg2_det.cam)
    
    def _call_back(value, timestamp, **kwargs):
        if pg2_det.cam.num_images.get() == value:
            _st._finished(success=True)
    
    pg2_det.cam.num_images_counter.subscribe(_call_back)
    
    while (pg2_det.cam.num_images.get() < pg2_det.cam.num_images_counter.get()): 
        yield from bps.sleep(time_wait)



def collect_background(n):
    """scan plan for white/dark background collection (no motor motion)"""    
    yield from bps.mv(pg2_det.cam.acquire, 0)
    
    for k,v in {
        "num_capture": n,
        "capture":     1,
    }.items(): pg2_det.tiff1.stage_sigs[k] = v
    
    for k,v in {
        "reset_filter":     1,
        "num_filter":       n_frames,
    }.items(): pg2_det.proc1.stage_sigs[k] = v
        
    for k,v in {
        "trigger_mode": "Internal",
        "image_mode":   "Multiple",
        "num_images":   n_frames,
    }.items(): pg2_det.cam.stage_sigs[k] = v
    
    @bpp.stage_decorator([pg2_det])
    @bpp.run_decorator()
#     @bpp.finalize_decorator(cleanup)  # finally block, cannot be empty function
    def scan():
        yield from bps.trigger_and_read([pg2_det])  #
    
#         if not test_mode: 
# #             yield from cam_io_wait()
#             yield from tiff_io_wait()
    
    return (yield from scan())
        

def collect_projections_alt():
    """translated directly from previous epics macro"""
#     yield from bps.mv(pg2_det.cam.capture, 0)  # stop the camera 
    
    # set staging paras
    # -- tiff1 plugin
    for k, v in {
        "num_capture": number_of_projections,     # how many images to disk (total)
        "capture":     1,                         # enable tiff1 plugin, 1 is "Capture"
    }.items(): pg2_det.tiff1.stage_sigs[k] = v
    
    # -- proc1 plugin
    for k, v in {
        "enable":           1,  # toggle on proc1
        "reset_filter":     1,  # reset number_filtered
        "num_filter":       n_frames,
    }.items(): pg2_det.proc1.stage_sigs[k] = v
    
    # -- cam plugin
    for k, v in {
        "num_images":   n_frames,      # one projection at a time
    }.items(): pg2_det.cam.stage_sigs[k] = v
   
    # now the scan step
    @bpp.stage_decorator([pg2_det])
    @bpp.run_decorator()
#     @bpp.finalize_decorator(cleanup)
    def scan_closure():
        for ang in np.linspace(omega_start, omega_end, number_of_projections):
            yield from bps.checkpoint()
            yield from bps.mv(preci, ang)
            yield from bps.trigger_and_read([pg2_det])
    
    return (yield from scan_closure())


# def collect_projections():
#     """scan plan for projection image collections (step scans)"""
    
#     yield from bp.scan(
#         [pg2_det], 
#         preci,        # rotary motor
#         omega_start, omega_end, number_of_projections,
#     )  # interanlly using trigger and wait, so it should wait for the image to finish transfer from cam to tiff

In [20]:
def tomo_scan():
    """
    The master plan pass to RE for
    
    1. pre-white-field background collection
    2. projection collection
    3. post-white-field background collection
    4. post-dark-field background collection
    """
    # prep, set the default values for detectors
    # NOTE:
    #    This part is run on the Jupyter end, and send to device
    #    during execution by RE
    for k, v in config_cam.items():     pg2_det.cam.stage_sigs[k]   = v
    for k, v in config_proc1.items():   pg2_det.proc1.stage_sigs[k] = v
    for k, v in config_tiff1.items():   pg2_det.tiff1.stage_sigs[k] = v
    
    # frist, n_white background
    yield from bps.mv(A_shutter, 'open')
    yield from bps.install_suspender(suspend_A_shutter)
    
    current_samx = samx.position
    yield from bps.mv(samx, current_samx + samOutDist)
    yield from collect_background(n_white)
    yield from bps.mv(samx, current_samx)
    
    # then, projection images
    yield from collect_projections_alt()
    
#     # now collect the post white field backgrounds
#     current_samx = samx.position
#     yield from bps.mv(samx, current_samx - samOutDist)
#     yield from collect_background(n_white)
#     yield from bps.mv(samx, current_samx)
    
#     # finally, collect the dark field images
    yield from bps.remove_suspender(suspend_A_shutter)
    yield from bps.mv(A_shutter, "close")
    yield from collect_background(n_dark)


Now pass the plan to summarizer for sanity check

In [21]:
from bluesky.simulators import summarize_plan

test_mode = True

summarize_plan(tomo_scan())

test_mode = False

A_shutter -> open
tomostage_samX -> -5.0
pg2_det_cam_acquire -> 0
=================================== Open Run ===================================
  Read ['pg2_det']
================================== Close Run ===================================
tomostage_samX -> 6.938893903907228e-18
=================================== Open Run ===================================
tomostage_preci -> -180.0
  Read ['pg2_det']
tomostage_preci -> -179.75
  Read ['pg2_det']
tomostage_preci -> -179.5
  Read ['pg2_det']
tomostage_preci -> -179.25
  Read ['pg2_det']
tomostage_preci -> -179.0
  Read ['pg2_det']
tomostage_preci -> -178.75
  Read ['pg2_det']
tomostage_preci -> -178.5
  Read ['pg2_det']
tomostage_preci -> -178.25
  Read ['pg2_det']
tomostage_preci -> -178.0
  Read ['pg2_det']
tomostage_preci -> -177.75
  Read ['pg2_det']
tomostage_preci -> -177.5
  Read ['pg2_det']
tomostage_preci -> -177.25
  Read ['pg2_det']
tomostage_preci -> -177.0
  Read ['pg2_det']
tomostage_preci -> -176.75
  Read ['pg2_d

tomostage_preci -> -95.75
  Read ['pg2_det']
tomostage_preci -> -95.5
  Read ['pg2_det']
tomostage_preci -> -95.25
  Read ['pg2_det']
tomostage_preci -> -95.0
  Read ['pg2_det']
tomostage_preci -> -94.75
  Read ['pg2_det']
tomostage_preci -> -94.5
  Read ['pg2_det']
tomostage_preci -> -94.25
  Read ['pg2_det']
tomostage_preci -> -94.0
  Read ['pg2_det']
tomostage_preci -> -93.75
  Read ['pg2_det']
tomostage_preci -> -93.5
  Read ['pg2_det']
tomostage_preci -> -93.25
  Read ['pg2_det']
tomostage_preci -> -93.0
  Read ['pg2_det']
tomostage_preci -> -92.75
  Read ['pg2_det']
tomostage_preci -> -92.5
  Read ['pg2_det']
tomostage_preci -> -92.25
  Read ['pg2_det']
tomostage_preci -> -92.0
  Read ['pg2_det']
tomostage_preci -> -91.75
  Read ['pg2_det']
tomostage_preci -> -91.5
  Read ['pg2_det']
tomostage_preci -> -91.25
  Read ['pg2_det']
tomostage_preci -> -91.0
  Read ['pg2_det']
tomostage_preci -> -90.75
  Read ['pg2_det']
tomostage_preci -> -90.5
  Read ['pg2_det']
tomostage_preci -> -9

tomostage_preci -> -7.75
  Read ['pg2_det']
tomostage_preci -> -7.5
  Read ['pg2_det']
tomostage_preci -> -7.25
  Read ['pg2_det']
tomostage_preci -> -7.0
  Read ['pg2_det']
tomostage_preci -> -6.75
  Read ['pg2_det']
tomostage_preci -> -6.5
  Read ['pg2_det']
tomostage_preci -> -6.25
  Read ['pg2_det']
tomostage_preci -> -6.0
  Read ['pg2_det']
tomostage_preci -> -5.75
  Read ['pg2_det']
tomostage_preci -> -5.5
  Read ['pg2_det']
tomostage_preci -> -5.25
  Read ['pg2_det']
tomostage_preci -> -5.0
  Read ['pg2_det']
tomostage_preci -> -4.75
  Read ['pg2_det']
tomostage_preci -> -4.5
  Read ['pg2_det']
tomostage_preci -> -4.25
  Read ['pg2_det']
tomostage_preci -> -4.0
  Read ['pg2_det']
tomostage_preci -> -3.75
  Read ['pg2_det']
tomostage_preci -> -3.5
  Read ['pg2_det']
tomostage_preci -> -3.25
  Read ['pg2_det']
tomostage_preci -> -3.0
  Read ['pg2_det']
tomostage_preci -> -2.75
  Read ['pg2_det']
tomostage_preci -> -2.5
  Read ['pg2_det']
tomostage_preci -> -2.25
  Read ['pg2_det']

tomostage_preci -> 80.5
  Read ['pg2_det']
tomostage_preci -> 80.75
  Read ['pg2_det']
tomostage_preci -> 81.0
  Read ['pg2_det']
tomostage_preci -> 81.25
  Read ['pg2_det']
tomostage_preci -> 81.5
  Read ['pg2_det']
tomostage_preci -> 81.75
  Read ['pg2_det']
tomostage_preci -> 82.0
  Read ['pg2_det']
tomostage_preci -> 82.25
  Read ['pg2_det']
tomostage_preci -> 82.5
  Read ['pg2_det']
tomostage_preci -> 82.75
  Read ['pg2_det']
tomostage_preci -> 83.0
  Read ['pg2_det']
tomostage_preci -> 83.25
  Read ['pg2_det']
tomostage_preci -> 83.5
  Read ['pg2_det']
tomostage_preci -> 83.75
  Read ['pg2_det']
tomostage_preci -> 84.0
  Read ['pg2_det']
tomostage_preci -> 84.25
  Read ['pg2_det']
tomostage_preci -> 84.5
  Read ['pg2_det']
tomostage_preci -> 84.75
  Read ['pg2_det']
tomostage_preci -> 85.0
  Read ['pg2_det']
tomostage_preci -> 85.25
  Read ['pg2_det']
tomostage_preci -> 85.5
  Read ['pg2_det']
tomostage_preci -> 85.75
  Read ['pg2_det']
tomostage_preci -> 86.0
  Read ['pg2_det']


  Read ['pg2_det']
tomostage_preci -> 163.75
  Read ['pg2_det']
tomostage_preci -> 164.0
  Read ['pg2_det']
tomostage_preci -> 164.25
  Read ['pg2_det']
tomostage_preci -> 164.5
  Read ['pg2_det']
tomostage_preci -> 164.75
  Read ['pg2_det']
tomostage_preci -> 165.0
  Read ['pg2_det']
tomostage_preci -> 165.25
  Read ['pg2_det']
tomostage_preci -> 165.5
  Read ['pg2_det']
tomostage_preci -> 165.75
  Read ['pg2_det']
tomostage_preci -> 166.0
  Read ['pg2_det']
tomostage_preci -> 166.25
  Read ['pg2_det']
tomostage_preci -> 166.5
  Read ['pg2_det']
tomostage_preci -> 166.75
  Read ['pg2_det']
tomostage_preci -> 167.0
  Read ['pg2_det']
tomostage_preci -> 167.25
  Read ['pg2_det']
tomostage_preci -> 167.5
  Read ['pg2_det']
tomostage_preci -> 167.75
  Read ['pg2_det']
tomostage_preci -> 168.0
  Read ['pg2_det']
tomostage_preci -> 168.25
  Read ['pg2_det']
tomostage_preci -> 168.5
  Read ['pg2_det']
tomostage_preci -> 168.75
  Read ['pg2_det']
tomostage_preci -> 169.0
  Read ['pg2_det']
to

Then pass it to RunEngine for data collection

In [22]:
RE(tomo_scan())

Transient Scan ID: 1     Time: 2019-04-15 12:16:05
Persistent Unique Scan ID: '5168d96f-cb10-4bb7-9065-d2c928c5d2ef'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 12:16:07.0 |
+-----------+------------+
generator tomo_scan ['5168d96f'] (scan num: 1)



Transient Scan ID: 2     Time: 2019-04-15 12:16:12
Persistent Unique Scan ID: '614d2919-9d8f-4819-ab2c-c20722ab96bc'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 12:16:15.1 |
|         2 | 12:16:16.0 |
|         3 | 12:16:16.8 |
|         4 | 12:16:17.7 |
|         5 | 12:16:18.6 |
|         6 | 12:16:19.4 |
|         7 | 12:16:20.3 |
|         8 | 12:16:21.1 |
|         9 | 12:16:21.9 |
|        10 | 12:16:22.8 |
|        11 | 12:16:23.6 |
|        12 | 12:16:24.5 |
|        13 | 12:16:25.3 |
A 'deferred pause' has been requested. The RunEngine will pause at the next checkpoint. To pause immediat

RunEngineInterrupted: 
Your RunEngine is entering a paused state. These are your options for changing
the state of the RunEngine:

RE.resume()    Resume the plan.
RE.abort()     Perform cleanup, then kill plan. Mark exit_stats='aborted'.
RE.stop()      Perform cleanup, then kill plan. Mark exit_status='success'.
RE.halt()      Emergency Stop: Do not perform cleanup --- just stop.


In [25]:
RE.resume??

In [26]:
RE._rewind??

In [23]:
RE.abort()

Aborting: running cleanup and marking exit_status as 'abort'...
+-----------+------------+
generator tomo_scan ['614d2919'] (scan num: 2)



[E 12:21:04.126 run_engine:1260] Run aborted
    Traceback (most recent call last):
      File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py36/lib/python3.6/site-packages/bluesky/run_engine.py", line 1153, in _run
        msg = self._plan_stack[-1].send(resp)
      File "<ipython-input-20-77fa1978308b>", line 28, in tomo_scan
        yield from collect_projections_alt()
      File "<ipython-input-19-6c2401d780aa>", line 118, in collect_projections_alt
        return (yield from scan_closure())
      File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py36/lib/python3.6/site-packages/bluesky/utils.py", line 1011, in dec_inner
        return (yield from plan)
      File "/home/beams/S6BM/opt/miniconda3/envs/bluesky_py36/lib/python3.6/site-packages/bluesky/preprocessors.py", line 939, in stage_wrapper
        return (yield from finalize_wrapper(inn

['5168d96f-cb10-4bb7-9065-d2c928c5d2ef',
 '614d2919-9d8f-4819-ab2c-c20722ab96bc']